# Морфология
В этом ноутбуке описана подготовка данных для задачи POS-tagging. А также пара простых моделей на keras, решающих данную задачу. Оригинальная задача и ноутбук есть в контесте: https://www.kaggle.com/c/rupos2018/overview

## Часть 1. Загрузка корпуса
Здесь мы прочитаем корпуса из csv и разложим их по спискам.

In [ ]:
# для совместимости со вторым питоном
from __future__ import print_function
import io

In [ ]:
# Имена файлов с данными.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls drive/MyDrive/NLP/ft_native_300_ru_wiki_lenta_lower_case.vec

drive/MyDrive/NLP/ft_native_300_ru_wiki_lenta_lower_case.vec


In [ ]:
TRAIN_FILENAME = "drive/MyDrive/NLP/pos_tagging/train.csv"
TEST_FILENAME = "drive/MyDrive/NLP/pos_tagging/test.csv"

In [ ]:
# Считывание файлов.
from collections import namedtuple
WordForm = namedtuple("WordForm", "word pos gram")

def get_sentences(filename, is_train):
    sentences = []
    with io.open(filename, "r", encoding='utf-8') as r:
        # Пропускаем заголовок
        next(r)
        sentence = [] # будем заполнять список предложений
        for line in r:
            # предложения отделены по '\n'
            if len(line.strip()) == 0:
                if len(sentence) == 0:
                    continue
                sentences.append(sentence)
                sentence = []
                continue
            if is_train:
                # Формат: индекс\tномер_в_предложении\tсловоформа\tPOS#Грамемы
                word = line.strip().split("\t")[2]
                pos = line.strip().split("\t")[3].split("#")[0]
                gram = line.strip().split("\t")[3].split("#")[1]
                sentence.append(WordForm(word, pos, gram))
            else:
                word = line.strip().split("\t")[2]
                sentence.append(word)
        if len(sentence) != 0:
            sentences.append(sentence)
    return sentences

In [ ]:
train = get_sentences(TRAIN_FILENAME, True)
test = get_sentences(TEST_FILENAME, False)

In [ ]:
train[0]

[WordForm(word='А', pos='CONJ', gram='_'),
 WordForm(word='ведь', pos='PART', gram='_'),
 WordForm(word='для', pos='ADP', gram='_'),
 WordForm(word='конкретных', pos='ADJ', gram='Case=Gen|Degree=Pos|Number=Plur'),
 WordForm(word='изделий', pos='NOUN', gram='Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur'),
 WordForm(word='зачастую', pos='ADV', gram='Degree=Pos'),
 WordForm(word='нужен', pos='ADJ', gram='Degree=Pos|Gender=Masc|Number=Sing|Variant=Brev'),
 WordForm(word='монокристалл', pos='NOUN', gram='Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing'),
 WordForm(word='не', pos='PART', gram='_'),
 WordForm(word='только', pos='PART', gram='_'),
 WordForm(word='крупный', pos='ADJ', gram='Case=Nom|Degree=Pos|Gender=Masc|Number=Sing'),
 WordForm(word=',', pos='PUNCT', gram='_'),
 WordForm(word='но', pos='CONJ', gram='_'),
 WordForm(word='и', pos='PART', gram='_'),
 WordForm(word='заданной', pos='VERB', gram='Aspect=Perf|Case=Gen|Gender=Fem|Number=Sing|Tense=Past|VerbForm=Part|Voice=Pass'),
 Wor

In [ ]:
# Выведем, что получилось
for wordform in train[0][:10]:
    print(wordform.word, '\t', wordform.pos, '\t', wordform.gram)

А 	 CONJ 	 _
ведь 	 PART 	 _
для 	 ADP 	 _
конкретных 	 ADJ 	 Case=Gen|Degree=Pos|Number=Plur
изделий 	 NOUN 	 Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur
зачастую 	 ADV 	 Degree=Pos
нужен 	 ADJ 	 Degree=Pos|Gender=Masc|Number=Sing|Variant=Brev
монокристалл 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
не 	 PART 	 _
только 	 PART 	 _


Для простоты далее будем использовать токены слов и POS-теги. Но чтобы определять грамматические значения нужно еще провести некоторые манипуляции с данными, описанные в оригинальном ноутубке. Мы же ограничимся только определением частей речи

## Часть 2. Подготовка эмбеддингов

Обычно в качестве признаков для обучения сеток используются словные эмбеддинги. Для этого можно скачать предобученные и сохранить их в матрицу, где в расположатся векторы эмбеддингах по индексам, соответсвующих слов

In [ ]:
#запомним все уникальные слова и POS-теги в корпусе
word_set = set()
pos_set = set()
for sent in train:
    for wordform in sent:
        word_set.add(wordform.word.lower())
        pos_set.add(wordform.pos)

In [ ]:
for word in list(word_set)[:10]: 
    print(word, end=', ')
print()
print(pos_set)

ведрами, оснащенного, идеалу, воровстве, крупный, упрекнуть, 2269, конкретными, оккупационных, нравятся, 
{'PRON', 'INTJ', 'PUNCT', 'X', 'ADV', 'ADP', 'NUM', 'NOUN', 'PART', 'CONJ', 'SYM', 'AUX', 'ADJ', 'PROPN', 'DET', 'SCONJ', 'VERB'}


In [ ]:
#Загрузите эмбеддинги c https://nlp.stanford.edu/projects/glove/ или другие, которые вам нравятся и пропишите путь к ним
import numpy as np

word_embeddings_path = 'drive/MyDrive/NLP/ft_native_300_ru_wiki_lenta_lower_case.vec'
word2idx = {}
word_embeddings = []
embedding_size = 300
#Загружаем эмбеддинги
with io.open(word_embeddings_path, 'r', encoding="utf-8") as f_em:
    for line in f_em:
        split = line.strip().split(" ")
        # Совсем короткие строки пропускаем
        if len(split) <= 2:
            continue
        # Встретив первую подходящую строку, фиксируем размер эмбеддингов
        if embedding_size is None:
            embedding_size = len(split) - 1
            # Также нициализируем эмбеддинги для паддингов и неизвестных слов
            word2idx["PADDING_TOKEN"] = len(word2idx)
            word_embeddings.append(np.zeros(embedding_size))

            word2idx["UNKNOWN_TOKEN"] = len(word2idx)
            word_embeddings.append(np.random.uniform(-0.25, 0.25, embedding_size))
        # После этого все эмбеддинги должны быть одинаковой длины
        if len(split) - 1 != embedding_size:
            continue
            
        # Если слова нет в корпусе, то не будем для него запоминать эмбеддинг        
        if (split[0] not in word_set):
            continue
        
        word_embeddings.append(np.asarray(split[1:], dtype='float32'))
        word2idx[split[0]] = len(word2idx)

word_embeddings = np.array(word_embeddings, dtype='float32')

In [ ]:
len(word_set & set(word2idx.keys()))

92618

In [ ]:
len(word_set)

98880

In [ ]:
len(word2idx)

92618

In [ ]:
word_embeddings.shape

(92618, 300)

Как-то эмбеддинги не сильно подходят для данного корпуса поэтому, просто инициализируем рандмно матрицу эмбеддингов при определении сетки. Вам же предлагается все-таки поискать подходящие эмбеддинги и использовать их при обучении.

## Часть 3. Подготовка данных
Теперь нам остается только пронумеровать все слова и POS-теги и можно переходить к обучению сеток.

In [ ]:
word_to_index = {'PAD' : 0, 'UNK' : 1}
for word in word_set:
    word_to_index[word] = len(word_to_index)

In [ ]:
pos_to_index = {}
index_to_pos = {}
for pos in pos_set:
    pos_to_index[pos] = len(pos_to_index)
    index_to_pos[len(index_to_pos)] = pos

In [ ]:
# для полносвязной сетки просто положим все индексы в один список
data_X = []
data_Y = []
for sent in train:
    for wordform in sent:
        data_X.append(word_to_index[wordform.word.lower()])
        data_Y.append(pos_to_index[wordform.pos])

In [ ]:
print(data_X[:10])
print(data_Y[:10])

[20541, 41209, 15563, 84569, 28007, 10201, 95436, 69436, 47276, 26882]
[9, 8, 5, 12, 7, 4, 12, 7, 8, 8]


## Часть 4. Полносвязная сеть
Самой простой моделью является обычный перцептрон. На вход сетки будем подавать просто эмдеддинг каждого слова, на выходе ожидать распредедение вероятностей по тегам. В качестве фреймворка достаточно будет использовать keras и его Sequential модель (https://keras.io/models/sequential/), в которую слои добавляются последовательно, с помощью метода `add`.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Activation, Flatten

In [ ]:
model = Sequential()
# на самом деле на вход сетки будет добавляться индекс слова, а слой эмбеддинга будет возвращать для него вектор
model.add(Embedding(input_length=1, input_dim=len(word_to_index), output_dim=50, embeddings_initializer='random_uniform',
                    trainable=False)) # матрицу эмбеддингов просто инициализируем нормальным распределением и отключим обучение
# далее нам нужно схлопнуть трехмерный тензор с одной фиктивной размерностью в двумерный
model.add(Flatten())
model.add(Dense(100)) # основной полносвязный слой
model.add(Activation('relu')) # для приличия добавим функцию активации
model.add(Dense(len(pos_to_index))) # выходной слой тоже полносвязный размерности по кол-ву тегов
model.add(Activation('softmax')) # ну и в конце делаем softmax, чтобы получить распределение
model.summary() # вывод получившейся модели

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 50)             4944100   
_________________________________________________________________
flatten (Flatten)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 100)               5100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                1717      
_________________________________________________________________
activation_1 (Activation)    (None, 17)                0         
Total params: 4,950,917
Trainable params: 6,817
Non-trainable params: 4,944,100
__________________________________________

In [ ]:
# компилируем модель
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# и обучаем
model.fit(np.array(data_X), np.array(data_Y), epochs=5, batch_size=256)

Epoch 1/5
3324/3324 [==============================] - 5s 2ms/step - loss: 1.1216 - accuracy: 0.6195
Epoch 2/5
3324/3324 [==============================] - 6s 2ms/step - loss: 1.0096 - accuracy: 0.6475
Epoch 3/5
3324/3324 [==============================] - 5s 2ms/step - loss: 0.9453 - accuracy: 0.6654
Epoch 4/5
3324/3324 [==============================] - 6s 2ms/step - loss: 0.9054 - accuracy: 0.6778
Epoch 5/5
3324/3324 [==============================] - 6s 2ms/step - loss: 0.8776 - accuracy: 0.6851


Проверка обученности модели остается за вами. Этот пример лишь для того, чтобы показать как собрать сетку и скормить ей данные.

## Часть 5. Рекуррентая сеть.

Далее рассмотрим более приближенную к SOTA модель. Ей является рекуррентая сеть, которая принимает эмбеддинги слов в предложении и генерирует для них распределение вероятностей. Основным отличием от прошлой в том, что теперь мы будем использовать соседние слова как раз за счет рекуррентого слоя. Для этой модели мы уже будем использовать функциональный способ задания модели все того же кераса (https://keras.io/models/model/).

In [ ]:
from keras.layers import LSTM, TimeDistributed,Bidirectional, Input
from keras.models import Model

In [ ]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(None,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(input_dim=len(word_to_index), output_dim=50, 
                             trainable=False, embeddings_initializer='random_uniform',
                             name='embedding')(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
blstm_layer = Bidirectional(LSTM(100, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(blstm_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          4944100   
_________________________________________________________________
blstm (Bidirectional)        (None, None, 200)         120800    
_________________________________________________________________
time_distributed (TimeDistri (None, None, 17)          3417      
Total params: 5,068,317
Trainable params: 124,217
Non-trainable params: 4,944,100
_________________________________________________________________


Далее нам нужно было бы распределить слова по предложениям, распределить по группам по длине, выравнить предложения по длине в одной групе, заполнив недостающие слова паддингами. Но это довольно неприятный процесс, а мне просто хочется запустить сетку и проверить, что она вообще работает, что сошлись все разверности. Поэтому просто раскидаем по 10 слов с помощью `numpy.reshape`

In [ ]:
rnnX = np.reshape(data_X[:850000], (-1,10))
rnnY = np.reshape(data_Y[:850000], (-1,10,1))

In [ ]:
np.shape(rnnX)

(85000, 10)

Ну и проверим, что оно вообще работает.

In [ ]:
model.fit(rnnX, rnnY, epochs=1, batch_size=256)

333/333 [==============================] - 26s 77ms/step - loss: 2.0785 - accuracy: 0.3580


## Часть 6. Задание
В качестве упражения предлагается довести до ума обучения второй модели: распределить слова по предложениям, написать тестирование модели и собственно посмотреть как оно обучилось. Тестировать предлагаю на последней 1000 предложений, обучать - на остальном. Кто уверен в своих желаниях, то может решить оригинальную задачу: предсказывать также грамматические категории. 

In [ ]:
pos_to_index['PAD'] = len(pos_to_index)

In [ ]:
# для RNN сетки соберем данные по предложениям
sent_len = 30
data_X = []
data_Y = []
for sent in train:
  x_sent_ind = []
  y_sent_ind = []
  for wordform in sent:
      x_sent_ind.append(word_to_index[wordform.word.lower()])
      y_sent_ind.append(pos_to_index[wordform.pos])
  # if small then padd
  if len(x_sent_ind) <= sent_len:
    x_sent_ind.extend([word_to_index['PAD']]*(sent_len-len(x_sent_ind)))
    y_sent_ind.extend([pos_to_index['PAD']]*(sent_len-len(y_sent_ind)))
    data_X.append(x_sent_ind)
    data_Y.append(y_sent_ind)
  # if big then cut
  else:
    for i in range(len(x_sent_ind) - sent_len):
      data_X.append(x_sent_ind[i:sent_len+i])
      data_Y.append(y_sent_ind[i:sent_len+i]) 


In [ ]:
data_Y = np.array(data_Y)
data_Y = data_Y.reshape((data_Y.shape[0], data_Y.shape[1], 1))
data_X = np.array(data_X)


In [ ]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(sent_len,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(input_dim=len(word_to_index), output_dim=50, 
                             trainable=False, embeddings_initializer='random_uniform',
                             name='embedding')(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
blstm_layer = Bidirectional(LSTM(100, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
hidden_layer = TimeDistributed(Dense(64, activation='relu', name='hidden'))(blstm_layer)
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(hidden_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 50)            4944100   
_________________________________________________________________
blstm (Bidirectional)        (None, 30, 200)           120800    
_________________________________________________________________
time_distributed (TimeDistri (None, 30, 64)            12864     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 18)            1170      
Total params: 5,078,934
Trainable params: 134,834
Non-trainable params: 4,944,100
_________________________________________________________________


In [ ]:
model.fit(data_X, data_Y, epochs=5, batch_size=256)

Epoch 1/5
360/360 [==============================] - 78s 217ms/step - loss: 1.6183 - accuracy: 0.4936
Epoch 2/5
360/360 [==============================] - 81s 226ms/step - loss: 1.1624 - accuracy: 0.6296
Epoch 3/5
105/360 [=======>......................] - ETA: 57s - loss: 1.0817 - accuracy: 0.6469

KeyboardInterrupt: ignored

Add embeddings

In [ ]:
word_embeddings.shape

(92618, 300)

In [ ]:
len(word2idx)

92618

In [ ]:
len(word_set)

98880

In [ ]:
unknown_tok = word_set - set(word2idx.keys())

In [ ]:
if "UNKNOWN_TOKEN" not in word2idx:
  word2idx["UNKNOWN_TOKEN"] = len(word2idx)
  unknown_emb = np.random.uniform(-0.25, 0.25, embedding_size)
  word_embeddings = np.vstack([word_embeddings, unknown_emb])

In [ ]:
len(word2idx)

92619

In [ ]:
word_embeddings.shape

(92619, 300)

In [ ]:
for word in unknown_tok:
    word2idx[word] = word2idx["UNKNOWN_TOKEN"]

In [ ]:
if "PADDING_TOKEN" not in word2idx:
  word2idx["PADDING_TOKEN"] = word2idx["UNKNOWN_TOKEN"] + 1
  word_embeddings = np.vstack([word_embeddings, np.zeros(embedding_size)])

In [ ]:
word2idx['PADDING_TOKEN']

92619

In [ ]:
# для RNN сетки соберем данные по предложениям
sent_len = 30
data_X = []
data_Y = []
for sent in train:
  x_sent_ind = []
  y_sent_ind = []
  for wordform in sent:
      x_sent_ind.append(word2idx[wordform.word.lower()])
      y_sent_ind.append(pos_to_index[wordform.pos])
  # if small then padd
  if len(x_sent_ind) <= sent_len:
    x_sent_ind.extend([word2idx['PADDING_TOKEN']]*(sent_len-len(x_sent_ind)))
    y_sent_ind.extend([pos_to_index['PAD']]*(sent_len-len(y_sent_ind)))
    data_X.append(x_sent_ind)
    data_Y.append(y_sent_ind)
  # if big then cut
  else:
    for i in range(len(x_sent_ind) - sent_len):
      data_X.append(x_sent_ind[i:sent_len+i])
      data_Y.append(y_sent_ind[i:sent_len+i]) 


In [ ]:
data_Y = np.array(data_Y)
data_Y = data_Y.reshape((data_Y.shape[0], data_Y.shape[1], 1))
data_X = np.array(data_X)

In [ ]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(sent_len,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(input_dim=word_embeddings.shape[0], output_dim=300, 
                             trainable=False, weights=[word_embeddings],
                             name='embedding')(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
blstm_layer = Bidirectional(LSTM(100, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
hidden_layer = TimeDistributed(Dense(64, activation='relu', name='hidden'))(blstm_layer)
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(hidden_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 300)           27786000  
_________________________________________________________________
blstm (Bidirectional)        (None, 30, 200)           320800    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 64)            12864     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 18)            1170      
Total params: 28,120,834
Trainable params: 334,834
Non-trainable params: 27,786,000
_________________________________________________________________


In [ ]:
model.fit(data_X[:-1000], data_Y[:-1000], epochs=10, batch_size=256)

Epoch 1/10
356/356 [==============================] - 102s 286ms/step - loss: 0.3433 - accuracy: 0.9017
Epoch 2/10
356/356 [==============================] - 102s 285ms/step - loss: 0.1159 - accuracy: 0.9638
Epoch 3/10
356/356 [==============================] - 101s 284ms/step - loss: 0.0855 - accuracy: 0.9734
Epoch 4/10
356/356 [==============================] - 102s 287ms/step - loss: 0.0680 - accuracy: 0.9791
Epoch 5/10
356/356 [==============================] - 103s 289ms/step - loss: 0.0567 - accuracy: 0.9828
Epoch 6/10
356/356 [==============================] - 102s 285ms/step - loss: 0.0484 - accuracy: 0.9855
Epoch 7/10
356/356 [==============================] - 101s 284ms/step - loss: 0.0419 - accuracy: 0.9875
Epoch 8/10
356/356 [==============================] - 101s 284ms/step - loss: 0.0372 - accuracy: 0.9890
Epoch 9/10
356/356 [==============================] - 101s 284ms/step - loss: 0.0330 - accuracy: 0.9903
Epoch 10/10
356/356 [==============================] - 101s 284m

In [ ]:
y_pred = model.predict(data_X[-1000:])

In [ ]:
np.reshape(data_Y[-1000:], (-1)).shape

(30000,)

In [ ]:
y_pred_classes = []
for i in range(y_pred.shape[0]):
  for j in range(sent_len):
    y_pred_classes.append(np.argmax(y_pred[i][j]))

In [ ]:
y_pred_classes = np.array(y_pred_classes)
y_pred_classes.shape

(30000,)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_pred_classes, np.reshape(data_Y[-1000:], (-1))))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92       784
           1       0.00      0.00      0.00         0
           2       0.99      0.98      0.99      3987
           3       0.00      0.00      0.00         1
           4       0.95      0.93      0.94      1074
           5       0.99      1.00      0.99      2077
           6       0.98      0.98      0.98       439
           7       0.99      0.98      0.99      5720
           8       0.94      0.90      0.92       732
           9       0.93      0.96      0.95       840
          11       0.85      0.97      0.90       159
          12       0.98      0.96      0.97      2459
          13       0.90      0.97      0.93       683
          14       0.92      0.97      0.94       564
          15       0.96      0.97      0.96       520
          16       0.97      0.97      0.97      2503
          17       1.00      1.00      1.00      7458

    accuracy              

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Make embedding trainable

In [ ]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(sent_len,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(input_dim=word_embeddings.shape[0], output_dim=300, 
                             trainable=True, weights=[word_embeddings],
                             name='embedding')(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
blstm_layer = Bidirectional(LSTM(100, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
hidden_layer = TimeDistributed(Dense(64, activation='relu', name='hidden'))(blstm_layer)
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(hidden_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 300)           27786000  
_________________________________________________________________
blstm (Bidirectional)        (None, 30, 200)           320800    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 30, 64)            12864     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 30, 18)            1170      
Total params: 28,120,834
Trainable params: 28,120,834
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(data_X[:-1000], data_Y[:-1000], epochs=10, batch_size=256)

Epoch 1/10
356/356 [==============================] - 184s 516ms/step - loss: 0.3269 - accuracy: 0.9104
Epoch 2/10
356/356 [==============================] - 181s 508ms/step - loss: 0.0378 - accuracy: 0.9884
Epoch 3/10
356/356 [==============================] - 180s 505ms/step - loss: 0.0194 - accuracy: 0.9942
Epoch 4/10
356/356 [==============================] - 180s 506ms/step - loss: 0.0107 - accuracy: 0.9969
Epoch 5/10
356/356 [==============================] - 180s 507ms/step - loss: 0.0060 - accuracy: 0.9984
Epoch 6/10
356/356 [==============================] - 180s 506ms/step - loss: 0.0036 - accuracy: 0.9990
Epoch 7/10
356/356 [==============================] - 179s 502ms/step - loss: 0.0023 - accuracy: 0.9994
Epoch 8/10
356/356 [==============================] - 178s 501ms/step - loss: 0.0015 - accuracy: 0.9996
Epoch 9/10
356/356 [==============================] - 178s 501ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 10/10
356/356 [==============================] - 179s 503m

In [ ]:
y_pred = model.predict(data_X[-1000:])

In [ ]:
y_pred_classes = []
for i in range(y_pred.shape[0]):
  for j in range(sent_len):
    y_pred_classes.append(np.argmax(y_pred[i][j]))

In [ ]:
print(classification_report(y_pred_classes, np.reshape(data_Y[-1000:], (-1))))

              precision    recall  f1-score   support

           0       0.94      0.86      0.90       814
           1       1.00      1.00      1.00         8
           2       0.99      0.98      0.98      3966
           3       0.00      0.00      0.00         1
           4       0.94      0.94      0.94      1060
           5       0.99      0.99      0.99      2097
           6       0.92      0.99      0.96       409
           7       0.98      0.97      0.98      5720
           8       0.92      0.89      0.90       734
           9       0.91      0.95      0.93       835
          11       0.88      0.97      0.92       165
          12       0.96      0.95      0.96      2435
          13       0.91      0.92      0.91       734
          14       0.85      0.96      0.90       528
          15       0.98      0.95      0.96       541
          16       0.96      0.96      0.96      2495
          17       1.00      1.00      1.00      7458

    accuracy              